In [25]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
#
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import MDS, Isomap, LocallyLinearEmbedding
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Imports para las redes
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Reshape, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [7]:
# Cargamos los datos
lfw_people = fetch_lfw_people(min_faces_per_person=53, resize=0.4)
X, y, images = lfw_people.images, lfw_people.target, lfw_people.images
X.shape, y.shape

((1456, 50, 37), (1456,))

In [8]:
# Normalizamos las im'agenes al rango [0,1]
X = X / 255.0

# Codificamos las etiquetas
labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(y)

# Dividimos los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1019, 50, 37), (437, 50, 37), (1019,), (437,))

In [9]:
# Escalamos las imagenes
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X.shape[1] * X.shape[2]))
X_test_scaled = scaler.transform(X_test.reshape(-1, X.shape[1] * X.shape[2]))
X_train_scaled.shape, X_test_scaled.shape

((1019, 1850), (437, 1850))

In [11]:
# Redimensionamos las imagenes para que tengan la forma adecuada para la CNN 
X_train_redm = X_train.reshape(-1, X.shape[1], X.shape[2], 1)
X_test_redm = X_test.reshape(-1, X.shape[1], X.shape[2], 1)
X_train_redm.shape, X_test_redm.shape

((1019, 50, 37, 1), (437, 50, 37, 1))

In [36]:
# Modelo CNN
modelo = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(X.shape[1], X.shape[2],1)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(labelEncoder.classes_), activation='softmax')
])
# Compilamos el modelo
modelo.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
#
modelo.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_33 (Conv2D)              │ (None, 48, 35, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 46, 33, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 23, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 21, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 19, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_31 (MaxPooling2D) │ (None, 9, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 9, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 6912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │       884,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,015,754 (3.87 MB)

 Trainable params: 1,015,754 (3.87 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
earlyStop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
modelo.fit(X_train_redm, y_train, epochs=100, batch_size=32 ,validation_split=.2, callbacks=[earlyStop])
#
test_loss, test_acc = modelo.evaluate(X_test_redm, y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.3345 - loss: 2.1800 - val_accuracy: 0.3627 - val_loss: 1.9769
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.3684 - loss: 1.9802 - val_accuracy: 0.3627 - val_loss: 1.9681
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.3809 - loss: 1.9467 - val_accuracy: 0.3627 - val_loss: 1.9774
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.3758 - loss: 1.9611 - val_accuracy: 0.3627 - val_loss: 1.9663
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.3730 - loss: 1.9462 - val_accuracy: 0.3627 - val_loss: 1.9660
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.3200 - loss: 2.0360 - val_accuracy: 0.3627 - val_loss: 2.0225
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.3900 - loss: 1.9367 - val_accuracy: 0.3627 - val_loss: 1.9631
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.3747 - loss: 1.9443 - val_accuracy: 0.